In [ ]:
import math
import numpy as np
import random

In [ ]:
class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __pow__(self, other):

    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out

  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward

    return out

  def sigmoid(self):
    x = self.data
    t = 1/(1+math.exp(-x))
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += t*(1-t) * out.grad
    out._backward = _backward

    return out

  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')

    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward

    return out

  def relu(self):
    out = Value(max(0, self.data), (self,), 'ReLU')

    def _backward():
        self.grad += (out.data > 0) * out.grad
    out._backward = _backward

    return out


  def backward(self):

    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()


In [34]:
class Neuron:

  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1), label=f'w{i+1}') for i in range(nin)]
    self.b = Value(random.uniform(-1,1), label='b')

  def __call__(self, x, act):
    # w * x + b
    out = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    if act:
      out = out.sigmoid()

    return out

  def parameters(self):
    return self.w + [self.b]

class Layer:

  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x, act):

    outs = [n(x, act) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:

  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
      for i, layer in enumerate(self.layers):
          if i < len(self.layers) - 1:
              x = layer(x, 1)
          else:
              x = layer(x, 0)
      return x

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
diabetes = load_diabetes()

In [50]:
X_reg = diabetes.data
y_reg = diabetes.target
print(X_reg[0], y_reg[0])


scaler = StandardScaler()

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

X_train_reg = scaler.fit_transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)



[ 0.03807591  0.05068012  0.06169621  0.02187239 -0.0442235  -0.03482076
 -0.04340085 -0.00259226  0.01990749 -0.01764613] 151.0


In [51]:
n = MLP(10, [10, 10, 10, 1])  # MLP con più livelli

# Learning rate
learning_rate = 0.1  # Tasso di apprendimento

# Allenamento
for epoch in range(30):  # Numero di epoch
    # Forward pass
    loss = 0
    leng = X_train_reg.shape[0]
    for i in range(leng):
      loss = loss + (n(X_train_reg[i]) - y_train_reg[i]) ** 2
    loss = loss/leng

    # Output della perdita

    print(f'Epoch {epoch + 1}, Loss: {loss.data}')

    # Backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    # Aggiornamento dei parametri
    for p in n.parameters():
        p.data -= learning_rate * p.grad


Epoch 1, Loss: 29612.478566310336
Epoch 2, Loss: 6824.2503829119605
Epoch 3, Loss: 6259.197236001357
Epoch 4, Loss: 6105.607130124098
Epoch 5, Loss: 6081.03616584469
Epoch 6, Loss: 6077.104445057513
Epoch 7, Loss: 6076.4742577277675
Epoch 8, Loss: 6076.372125700567
Epoch 9, Loss: 6076.3543700117825
Epoch 10, Loss: 6076.350006109334
Epoch 11, Loss: 6076.347666495178
Epoch 12, Loss: 6076.345518648401
Epoch 13, Loss: 6076.343252928669
Epoch 14, Loss: 6076.340800495477
Epoch 15, Loss: 6076.338127546826
Epoch 16, Loss: 6076.335201612916
Epoch 17, Loss: 6076.331985111796
Epoch 18, Loss: 6076.3284329578
Epoch 19, Loss: 6076.324490374261
Epoch 20, Loss: 6076.320090129693
Epoch 21, Loss: 6076.315148814118
Epoch 22, Loss: 6076.309561713434
Epoch 23, Loss: 6076.30319562281
Epoch 24, Loss: 6076.295878570769
Epoch 25, Loss: 6076.2873847998835
Epoch 26, Loss: 6076.277412262798
Epoch 27, Loss: 6076.265547937859
Epoch 28, Loss: 6076.251212606788
Epoch 29, Loss: 6076.233569552975
Epoch 30, Loss: 6076.2

In [52]:

leng = X_test_reg.shape[0]
loss = 0
for i in range(leng):
  loss = loss + ((n(X_test_reg[i]) - y_test_reg[i]) ** 2)/leng
loss

Value(data=5361.363993291448)